In [ ]:
class FeatureSelection(object):
    
    #documentation 

    def __init__(self):
        self.poscutoff = 0.5
        self.negcutoff = -0.5
        self.model = Ridge()
        self.trainsubset = None #is there a better way to initialize these self attributes?
        self.trainsubset1 = None
        self.testsubset = None
        self.featurenames = None
        self.testpredictions = None
        self.localtime = time.asctime( time.localtime(time.time()) )
    
    def find_features(self, X, poscutoff, negcutoff):
        self.poscutoff = poscutoff
        self.negcutoff = negcutoff
        if isinstance(X, pd.DataFrame): X0 = X.corr().iloc[:,-1]
        elif isinstance(X, list): raise TypeError("Please input a pd.DataFrame object of the full House dataset with target variable in last column")
        elif isinstance(X, np.ndarray): raise TypeError("Please input a pd.DataFrame object of the full House dataset with target variable in last column")
        dict1 = {}
        for idx, i in enumerate(X0):
            if i > poscutoff:
                dict1[X0.index[idx]] = i
            elif i < negcutoff:
                dict1[X0.index[idx]] = i
            else: 
                continue
        self.featurenames = list(dict1.keys())[:-1]
        self.trainsubset = X[self.featurenames] #is there a way to call the self.preprocess method here to fillna the NaN values of the training data? 
        self.trainsubset1 = self.preprocess(self.trainsubset) #look into this 
        print("With positive and negative correlation cutoff values of %.2f and %.2f" %(poscutoff, negcutoff), "the selected features are: \n") 
        print(sorted(dict1.items(), key = lambda x: x[1], reverse = True)[1:])
        return self.trainsubset 
    
    def set_model(self, *args):
        #Model to try: LinearRegression, Ridge, Lasso, HuberRegressor, ElasticNet, BayesianRidge, DecisionTreeRegressor,
        #AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
        if 'Ridge' in args:
            self.model = Ridge()
            return self.model
        elif 'Lasso' in args: 
            self.model = Lasso() 
            return self.model
        elif 'ElasticNet' in args:
            self.model = ElasticNet()
            return self.model
        elif 'HuberRegressor' in args:
            self.model = HuberRegressor()
            return self.model
        elif 'BayesianRidge' in args:
            self.model = BayesianRidge()
            return self.model
        elif 'DecisionTreeRegressor' in args:
            self.model = DecisionTreeRegressor()
            return self.model
        elif 'AdaBoostRegressor' in args:
            self.model = AdaBoostRegressor()
            return self.model
        elif 'GradientBoostingRegressor' in args:
            self.model = GradientBoostingRegressor()
            return self.model
        elif 'RandomForestRegressor' in args:
            self.model = RandomForestRegressor()
            return self.model
        else:
            return self.model
        
    def tuning(self):
        paramgrid = {}
        print(self.model)
        if self.model == Ridge():
            paramgrid['alpha'] = np.linspace(0,1,10)
            gridsearch1 = GridSearchCV(self.model, )
            return True
        else:
            return False
    
    def fit(self, Ytrain):
        self.model.fit(self.trainsubset, Ytrain)
        print('\nThe train R^2 for %s with these features is:'%(self.model), '\n', self.model.score(self.trainsubset, Ytrain))
    
    def predict(self, Xtest):
        self.testpredictions = self.model.predict(self.testsubset)
        return self.testpredictions
    
    def preprocess(self, Xtest):
        if isinstance(Xtest, list): raise TypeError("Xtest needs to be a DataFrame object")
        elif isinstance(Xtest, np.ndarray): raise TypeError()
        self.testsubset = Xtest[self.featurenames]
        self.testsubset = self.testsubset.fillna(self.testsubset.mean())
        self.trainsubset = self.trainsubset.fillna(self.trainsubset.mean())

    def to_csv1(self):
        testpredictionsdf = pd.DataFrame(self.testpredictions, index = range(1461, 2920), columns = ['SalePrice'])
        testpredictionsdf.index.name = 'Id'
        print(testpredictionsdf.isna().sum())
        #testpredictionsdf.to_csv('Kaggle Submission %s .csv'%(self.localtime))


x1 = FeatureSelection()
x1.set_model('Ridge')
x1.find_features(trainh, poscutoff = 0.1, negcutoff = -0.05) #why do I need to find_features before calling my fit function? #because you are calling self.trainsubset in this fit method, which comes from find_features. If you don't call it, self.trainsubset is None per the init  
x1.preprocess(testh) #how do I cut this out? self.preprocess(Xtest) in the predict method does not seem to work 
x1.fit(target)
x1.preprocess(testh) #how do I cut this out? self.preprocess(Xtest) in the predict method does not seem to work 
x1.predict(testh)
x1.tuning()
x1.to_csv1()
